#### Execute this cell before running any code in this file...

In [ ]:
# Import Javascript Libraries: Execute this cell...
from IPython.display import Javascript

display(Javascript("jupyter_lib.js"))

# Section 7: Beginnings of a Game

Goals:
 - Understand the base code for the game, and key components of a game at a higher level.

We have learned how to draw items to the screen, but this raises a simple question: How do we move objects on screen?

In video games, we use the same trick as in film, where we show images with objects slightly moved between frames. By showing the frames rapidly (~30-60 frames a second, 24 for film), what initially looks like separate frames becomes fluid motion to the human eye. This gives the 'illusion' of motion.

![Image of a circle moving a small amount several times...](./images/AnimationExample.png)

Animation is done by the game loop. The game loop, as the name implies, executes indefinitely until the game is terminated. The game loop can be broken into two parts:
 - `update`: Update the locations of all loaded objects in the game. (Move them).
 - `draw`: Draw the current state of the game to the screen.
 
This game loop runs at the same speed as the monitor refresh rate. Most monitors refresh at 60 frames per second.
 
To begin, we can write a very simple 'game' to showcase this.

In [ ]:
%%javascript

function update(timeStep, gameState) {
    gameState.timeStep = timeStep;
    
    // Add cube movement here!
}

function draw(canvas, painter, gameState) {
    // Clear the canvas...
    painter.fillStyle = "white"
    painter.fillRect(0, 0, canvas.width, canvas.height);
    
    // Example draw...
    painter.fillStyle = "black";
    painter.font = "30px Ariel";
    painter.fillText("Hello World!, Time Step: " + gameState.timeStep, 0, 30);
    painter.fillText("Mouse Pressed: " + gameState.mouse.pressed, 0, 60);
    painter.fillText("Mouse Location: " + JSON.stringify(gameState.mouse.location), 0, 90);
    painter.fillText("Keys Pressed: " + JSON.stringify(gameState.keysPressed), 0, 120);
    
    // Draw cube here!
}

// This function will get called over and over...
function gameLoop(timeStep, gameState) {
    update(timeStep, gameState);
    draw(gameState.canvas, gameState.painter, gameState);
}

element.makeBaseGame(gameLoop);

 As seen above, `element.makeBaseGame` accepts a function, that is executed forever until the game window is closed or terminated in some fashion. It passes two arguments to the function, the time step and game state object. The time step is the number of milliseconds that have passed between this and the last call to the loop, and the game state object can be used to stash the current state of the game.
 
As you may have noticed already, the makeBaseGame method adds some properties to the gameState object for general use. These include:
 - `lastTimeStamp`: Last time game was updated before this update.
 - `keepRunning`: Boolean flag to indicate if game should keep looping. Should be set to false to stop the game.
 - `canvas`: The canvas.
 - `painter`: The canvas painter or 2D rendering context.
 - `keysPressed`: An object which stores the key codes of all keys currently pressed down. You will use this for handling input.
 - `camera`: The camera created for the game. Converts game coordinates into screen coordinates, and can track an object.
 - `mousePressed`: If the mouse button is pressed down.
 - `mouseLocation`: The location of the mouse.
 
__Challenge Exercise: Add a cube to the game that starts in the center. Then make it move the the right each time the loop runs. (BONUS: Use keyboard input to determine input direction)__

## The Game Engine/Level Editor.

Since you have limited time in this class, you won't be implementing a game from scratch. Rather, you'll be using and building upon a game engine that will manage some more complex things for you. This includes:

 - Handling the storage and loading of "maps" from a file.
 - Handling of camera movement and chunk loading/unloading.
 - Detection and correction of collisions for certain objects.
 
and several other tasks that would take too long to implement in this class. 

For this engine, your entire game will be made up of game objects, or object types that extend the `GameObject` class. `GameObject`s can be:
 - Stored
 - Loaded
 - Created
 - Updated
 - Drawn
 
`GameObject` provides an API for doing each one of these tasks. You may wonder which you need to implement. 

There are 4 or 5 methods you have to implement, in order for an object to work in game. These include:
 - `constructor`: To set extra properties when the object is created.
 - `update`: Update the location of the object.
 - `draw`: Draw the object to screen.
 - `drawPreview`: Draw a preview of the object to a passed box. Used in the level editor.
 - `getHitBoxes`: Get the hit boxes of an object. Only need to override if an entity that is not 1 block by 1 block in size. Also needed for collision detection, which we will discuss later.
 - `getZOrder`: Get the z-order of this object. Object with a higher z-order are drawn on top of other objects.
 
As part of this class you will learn how to implement all of these. There is one additional feature that will need to be mentioned before we can begin using the game engine. Like several other engines/games, this engine splits objects into 3 types. 

 - `Blocks`: Represents a non-moving object that takes up a single block space on the grid. Typically used for the landscape since it shouldn't move.
 - `Entities`: Moving objects, not fixed to the grid, and can be any size. Entities represent enemies/player/etc.
 - `Player`: A special entity, as the camera is configured to track it.
 
You will need/want to implement one of all 3 types.
As can be seen below, you will fill in your code in the semi-filled out code below, which provides, 1 block, 1 entity, and 1 player, and then triggers the level editor... 

In [ ]:
%%javascript

class Player extends GameObject {
    constructor(x, y, assets) {
        super(x, y, assets);
        // More initialization here...
    }
    
    update(timeStep, gameState) {
        // Update code here. Returning true resets the entire level.
    }
    
    draw(canvas, painter, camera) {
        // Draw code here...
    }
    
    drawPreview(canvas, painter, box) {
        // Draw preview code here...
    }
}

class Entity extends GameObject {
    constructor(x, y, assets) {
        super(x, y, assets);
        // More initialization here...
    }
    
    update(timeStep, gameState) {
        // Update code here. Returning true destroys the enemy.
    }
    
    draw(canvas, painter, camera) {
        // Draw code here...
    }
    
    drawPreview(canvas, painter, box) {
        // Draw preview code here...
    }
}

class Block extends GameObject {
    constructor(x, y, assets) {
        super(x, y, assets);
        // More initialization here...
    }
    
    update(timeStep, gameState) {
        // Update code here. Returning true destroys the block.
    }
    
    draw(canvas, painter, camera) {
        // Draw code here...
    }
    
    drawPreview(canvas, painter, box) {
        // Draw preview code here...
    }
}

// We'll learn about this later....
let gameInfo = {
    objects: {
        blocks: [Block],
        entities: [Entity],
        players: [Player]
    },
    zones: {
        main: {
            zoneData: "levels/test_level.json"
        }
    },
    assets: {
        sprites: {},
        sounds: {}
    }
};

// First, we have to make a level.
element.levelEditor(gameInfo, "main");

One final note: The game engine adds some additional methods to the gameState. These include:
 - `addEntity(entity)`: Adds an entity to the game.
 - `addBlock(block)`: Adds a block to the game.
 - `getPlayers()`: Get the players in the game.
 - `getNeighboringBlocks(block)`: Get all of the blocks neighboring a given block.
 - `getEntities()`: Get all of the entities currently loaded on screen.
 
We won't worry about the specifics of these, and we will come back to them later. For now, you may notice when you open the above code in the level editor, your objects are non-existent! In the next lesson, we will learn how to draw the entities to the game's world.